In [1]:
%%time
# Name: Mohammed
import operator
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from mpl_toolkits.mplot3d import Axes3D
import itertools
from math import ceil
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import decomposition
from scipy.stats import zscore
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import random
from collections import Counter
from scipy.signal import butter, lfilter
from scipy.signal import freqz
from sklearn.preprocessing import robust_scale

def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut=4, highcut=30, fs=250, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def correlate(channel_data1, channel_data2):
    x = channel_data1
    y = channel_data2X = X[y!=0]
# y = y[y!=0]
    x = MAV_segments(x,10)
    y = MAV_segments(y,10)
    xc = np.correlate(x, y, mode='full')
    xc /= xc[xc.argmax()]
    return xc

def accuracy(y_pred, y_test):
    return 1 - np.linalg.norm(np.array(y_pred) - np.array(y_test),ord = 0)/float(len(y_pred))

def relabel_(Y):
    for i, label_ in enumerate(Y):
        if label_ == 0:
            if i != 0:
                if Y[i-1] == 1:
                    Y[i] = 1
                if Y[i-1] == 2:
                    Y[i] = 2
                if Y[i-1] == 3:
                    Y[i] = 3
                if Y[i-1] == 10:
                    Y[i] = random.choice([10])
            else:
                Y[i] = 10
    return Y

def switch_label(Y,a,b):
    for i, label_ in enumerate(Y):
        if label_ == a:
            Y[i] = b
        elif label_ == b:
            Y[i] = a
    return Y
            
def relabel__(Y):
    Y = switch_label(Y,3,4)
    for i, label_ in enumerate(Y):
        if label_ == 0:
            if i != 0:
                if Y[i-1] == 3:
                    Y[i] = 10
            else:
                Y[i] = 10
    return Y

def load_data(path):
    allFiles = glob.glob(path + "/new_data_main*.txt")
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
    frame = pd.concat(list_,ignore_index = True)
    return frame

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,"{0:.2f}".format(cm[i, j]) ,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def TD(channel_data):
    features = [MAV(channel_data)] + MAV_segments(channel_data) + diff_MAV(channel_data) + [ZC(channel_data)] +\
    [SSC(channel_data)] + [WL(channel_data)]
    return features

def MAV(channel_data):
    return sum(map(abs,channel_data))/len(channel_data)

def segment_window(channel_data, n_segments = 5.):
    seg_length = int(ceil(len(channel_data)/n_segments))
    segmented_data = [channel_data[x:x+seg_length] for x in range(0,len(channel_data),seg_length)]
    return segmented_data

def MAV_segments(channel_data, n_segments = 5.):
    segmented_data = segment_window(channel_data, n_segments)
    return map(MAV,segmented_data)

def diff_MAV(channel_data):
    segmented_data = segment_window(channel_data)
    prev_segments = segmented_data[:-1]
    next_segments = segmented_data[1:]
    prev_segments_MAV = map(MAV,prev_segments)
    next_segments_MAV = map(MAV,next_segments)
    return map(operator.sub, next_segments_MAV, prev_segments_MAV)

def ZC(channel_data, threshold = 10):
    prev_sample = channel_data[:-1]
    next_sample = channel_data[1:]
    left_side = map(abs,map(operator.sub, next_sample, prev_sample))
    right_side = map(abs,map(operator.add, next_sample, prev_sample))
    res = map(operator.sub, left_side, right_side)
    res = [1 if (x >= 0 and left > threshold) else 0 for x,left in zip(res,left_side)]
    return sum(res)/float(len(prev_sample)+1)

def SSC(channel_data, threshold = 5):
    the_sample = channel_data[1:-1]
    prev_sample = channel_data[:-2]
    next_sample = channel_data[2:]
    res = [1 if ((x > max(x_prev,x_next) or x < min(x_prev,x_next)) and max(abs(x_next - x),abs(x - x_prev)) > threshold)\
           else 0 for x,x_prev,x_next in zip(the_sample,prev_sample,next_sample)]
    return sum(res)/float(len(prev_sample)+2)

def WL(channel_data):
    prev_sample = channel_data[:-1]
    next_sample = channel_data[1:]
    diff = map(abs,map(operator.sub, next_sample, prev_sample))
    return sum(diff)/float(len(prev_sample)+1)
    
def SPM_features(data,domain,req_freq,width):
    new_mag = []
    new_angle = []
    mag = np.abs(data)
    angle = np.angle(data)
    for freq in req_freq:
        mag_agg = []
        angle_agg = []
        for i in range(0,len(mag)):
            if domain[i] >= (freq - width/2.) and  domain[i] <= (freq + width/2.):
                mag_agg.append(mag[i])
                angle_agg.append(angle[i])
        new_mag.append(sum(mag_agg)/float(len(mag_agg)))
        new_angle.append(sum(angle_agg)/float(len(angle_agg)))
    return new_mag#+new_angle
        
def SPM(channel_data, freq = 250):
    width = 0.5
    z = np.fft.rfft(channel_data) # FFT
    y = np.fft.rfftfreq(len(channel_data), d = 1./freq) # Frequency data
    #z = zscore(z)
    req_freq = [10, 12, 15, 20, 30]#np.arange(2,25,width)
    return SPM_features(z,y,req_freq,width)

def make_features(channel_data):
    return SPM(channel_data)

def opt_clf(X, y, params, clf, key, key2=None,key2_arg=None, key3=None,key3_arg=None, class_names_d = None, class_names = None):
    acc_log = []
    args = {}
    for lambda_ in params:
        args[key] = lambda_
        if key2 == None:
            classifier = clf(**args)
        else:
            for i, k in enumerate(key2):
                args[k] = key2_arg[i]
            classifier = clf(**args)
        # Split the data into a training set and a test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random.randint(0,100))
        # Run classifier
        y_pred = classifier.fit(X_train, y_train).predict(X_test)
        y_pred_train = classifier.fit(X_train, y_train).predict(X_train)
        print 'Accuracy for {}: Testing {}, Training {}'.format(lambda_, accuracy(y_test,y_pred), accuracy(y_train,y_pred_train))
        acc_log.append(accuracy(y_test,y_pred))
        
    axes = plt.gca()
    axes.scatter(params,acc_log)
    axes.set_xlim(min(params)-1,max(params)+1)
    # training with the best params
    max_index, max_value = max(enumerate(acc_log), key=operator.itemgetter(1))
    param = params[max_index]
    args[key] = param
    classifier = clf(**args)
    scores = cross_val_score(classifier, X, y, cv=5,n_jobs = -1)
    print "Cross-validation scores", scores
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random.randint(0,100))
    # Run classifier
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    y_pred = [x if x <= 6 else 0 for x in y_pred]
    y_test = [x if x <= 6 else 0 for x in y_test]
    # Compute confusion matrix
    if class_names_d is None:
        class_names_d = ['10Hz', '12Hz', '15Hz']
    if class_names is None:
        class_names = [2, 3, 4]
    cnf_matrix = confusion_matrix(y_test, y_pred, labels = class_names)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names_d,
                          title='Confusion matrix, without normalization')
    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names_d, normalize=True,
                          title='Normalized confusion matrix')
    return classifier

CPU times: user 596 ms, sys: 524 ms, total: 1.12 s
Wall time: 1.58 s


In [2]:
%%time
fs = 250
lowcut = 2
highcut = 35

# loading the data
path =r'data_ssvep2' # use your path
frame = load_data(path)

# data pre-processing
frame.sort(columns='time', axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
frame.reset_index(inplace = True)
del frame['index']
label = frame['label']
start_end = []
list1 = np.arange(0,frame.shape[0],50)
starts = list1[:-10]
ends = list1[10:]
for i, start in enumerate(starts):
    c = Counter(frame.label[start:ends[i]])
    if len(set(list(c.elements()))) == 1:
        label = c.most_common()[0][0]
    else:
        label = 10
    start_end.append((start,ends[i],label,frame.time[ends[i]]))
data_points = []

# for tup in start_end:
#     data_points.append([frame.chan1[tup[0]:tup[1]].tolist(),frame.chan2[tup[0]:tup[1]].tolist(),\
#                         frame.chan3[tup[0]:tup[1]].tolist(),frame.chan4[tup[0]:tup[1]].tolist(),\
#                         frame.chan5[tup[0]:tup[1]].tolist(),frame.chan6[tup[0]:tup[1]].tolist(),\
#                         frame.chan7[tup[0]:tup[1]].tolist(),frame.chan8[tup[0]:tup[1]].tolist(),tup[2]])

# tup = (0,200,1)
# data_points.append([frame.chan1[tup[0]:tup[1]].tolist(),frame.chan2[tup[0]:tup[1]].tolist(),\
#                     frame.chan3[tup[0]:tup[1]].tolist(),frame.chan4[tup[0]:tup[1]].tolist(),\
#                     frame.chan5[tup[0]:tup[1]].tolist(),frame.chan6[tup[0]:tup[1]].tolist(),\
#                     frame.chan7[tup[0]:tup[1]].tolist(),frame.chan8[tup[0]:tup[1]].tolist(),tup[2]])

for tup in start_end:
    data_points.append([frame.chan7[tup[0]:tup[1]].tolist(),frame.chan8[tup[0]:tup[1]].tolist(), tup[2]])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


CPU times: user 22.5 s, sys: 644 ms, total: 23.2 s
Wall time: 23 s


In [59]:
%%time
# feature extraction
data_ = []
Y_ = []
for i, data_point in enumerate(data_points):
    data_.append(map(make_features,data_point[0:2]))
    Y_.append(data_point[2])

CPU times: user 37 s, sys: 0 ns, total: 37 s
Wall time: 37 s


In [60]:
%%time
data_ = np.array(data_)
shape = np.shape(data_)
data = np.reshape(data_,(shape[0],shape[1]*shape[2]))

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 79.3 ms


In [61]:
%%time
Y__ = Y_[:]
# Y = relabel(Y__)
Y = relabel__(Y__)
# Y = Y__[:]
# X = robust_scale(data[~np.isnan(data).any(axis=1)],axis=1)
X = data[~np.isnan(data).any(axis=1)]
y = np.array(Y)[~np.isnan(data).any(axis=1)]

CPU times: user 48 ms, sys: 12 ms, total: 60 ms
Wall time: 38.3 ms


In [63]:
%%time
trees = [100,500,1000,2000]
layers = [(10,),(20,),(30,),(40,),(50,),(100,)]
reg = [1]
neighbors = [1,2,3,4,5,6]
Cs = [40,45,50,55,60,70,80,90]
depth = [15]
max_f = [20,20,20]
mask = (y != 10) & (y != 0) & (y != 1) & (y != 1)
X_new = X[mask]
y_new = y[mask]
clf = opt_clf(X_new[:], y_new[:], Cs, LogisticRegression, 'C',['penalty','class_weight'],['l1','balanced'],
              class_names_d = ['10Hz', '12Hz', '15Hz', '30Hz'], class_names = list(set(y_new)))
# clf = opt_clf(X_new[:], y_new[:], depth, RandomForestClassifier, 'max_depth',['n_jobs','class_weight','n_estimators','max_features'],[-1,'balanced',20,6])
# clf = opt_clf(X_new, y_new, reg, MLPClassifier, 'alpha',['hidden_layer_sizes', 'alpha'],[(500,6), 1])
# clf = opt_clf(X_new, y_new, Cs, SVC, 'C',['class_weight','kernel'],['balanced','rbf'])
plt.show()

Accuracy for 40: Testing 0.459620394492, Training 0.477573050437
Accuracy for 45: Testing 0.484555266096, Training 0.473044233513
Accuracy for 50: Testing 0.464458503908, Training 0.477511011849
Accuracy for 55: Testing 0.478600669892, Training 0.475029468329
Accuracy for 60: Testing 0.467621883141, Training 0.475091506917
Accuracy for 70: Testing 0.477298101972, Training 0.481791674421
Accuracy for 80: Testing 0.477298101972, Training 0.481853713009
Accuracy for 90: Testing 0.473576479345, Training 0.474533159625
Cross-validation scores [ 0.51  0.5   0.38  0.38  0.36]
Confusion matrix, without normalization
[[661 108 154  58  63]
 [ 84 736  95  47  53]
 [ 95  72 841  36  36]
 [444 212 182 169 121]
 [320 297 217  91 182]]
Normalized confusion matrix
[[ 0.63  0.1   0.15  0.06  0.06]
 [ 0.08  0.73  0.09  0.05  0.05]
 [ 0.09  0.07  0.78  0.03  0.03]
 [ 0.39  0.19  0.16  0.15  0.11]
 [ 0.29  0.27  0.2   0.08  0.16]]
CPU times: user 26.8 s, sys: 4.55 s, total: 31.4 s
Wall time: 35.2 s


In [124]:
import cPickle
# save the classifier
with open('LR_classifier.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)    

# load it again
with open('LR_classifier.pkl', 'rb') as fid:
    clf_loaded = cPickle.load(fid)

In [53]:
from scipy.fftpack import fft
# Number of sample points
N = 1356
# sample spacing
T = 1.0 / 250.0
x = np.linspace(0.0, N*T, N)
y = frame[start_end[1][0]:start_end[1][1]].chan1
yf = fft(y)
xf = np.linspace(0.0, 1.0/(16*T), N/16)
plt.plot(xf, 2.0/N * np.abs(yf[0:N/2])[0:N/16])
plt.grid()
plt.show()
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]
X_new = map(reject_outliers,X.reshape(X.shape[1],X.shape[0]))
X_new.shape

AttributeError: 'list' object has no attribute 'shape'